In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np

import vega_sim.api.data_raw as data_raw
import vega_sim.proto.vega as vega_protos
from vega_sim.grpc.client import VegaTradingDataClientV2

In [ ]:
data_client = VegaTradingDataClientV2('darling.network:3007')
market_id= "2dca7baa5f7269b08d053668bca03f97f72e9a162327eebd941c54f1f9fb8f80"
start = datetime.datetime(2023,5,31,12,57,45, tzinfo=datetime.timezone.utc)
end = start + datetime.timedelta(minutes=1)

trades = data_raw.get_trades(
    market_id=market_id,
    start=start,
    end=end,
    data_client=data_client
    )
market_data = data_raw.market_data_history(
    market_id=market_id,
    start=start,
    end=end,
    data_client=data_client
    )


In [ ]:
fig = plt.figure()

# prices
prices = np.array([float(t.price)*1e-1 for t in trades])
plt.plot([datetime.datetime.fromtimestamp(t.timestamp // 1000000000) for t in trades], prices, label = "price")
fig.autofmt_xdate()

# price monitoring bounds
ts = [datetime.datetime.fromtimestamp(d.timestamp // 1000000000) for d in market_data]
max_1 =  np.array([float(d.price_monitoring_bounds[0].max_valid_price)*1e-1 for d in market_data])
max_2 =  np.array([float(d.price_monitoring_bounds[1].max_valid_price)*1e-1 for d in market_data])
max_3 =  np.array([float(d.price_monitoring_bounds[2].max_valid_price)*1e-1 for d in market_data])
min_1 =  np.array([float(d.price_monitoring_bounds[0].min_valid_price)*1e-1 for d in market_data])
min_2 =  np.array([float(d.price_monitoring_bounds[1].min_valid_price)*1e-1 for d in market_data])
min_3 =  np.array([float(d.price_monitoring_bounds[2].min_valid_price)*1e-1 for d in market_data])

plt.plot(ts, max_1, label = "max (trigger 1)")
plt.plot(ts, max_2, label = "max (trigger 2)")
plt.plot(ts, max_3, label = "max (trigger 3)")
plt.plot(ts, min_1, label = "min (trigger 1)")
plt.plot(ts, min_2, label = "min (trigger 2)")
plt.plot(ts, min_3, label = "min (trigger 3)")


plt.legend()
plt.show()

In [ ]:
outlier_trades =  np.array(trades)[np.where(prices<np.max(np.maximum(min_1,min_2,min_3)))]
[vega_protos.vega._TRADE_TYPE.values_by_number[t.type].name for t in outlier_trades]